<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/lease_rests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install censusgeocode
!pip install rets
import csv
import pandas as pd
import censusgeocode as cg
import numpy as np
import sys
from rets import Session

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def lease_findifNull(lo_left,lo_right,la_left,la_right,research,zip,a):
  result = research.search(resource='Property', resource_class='Property', dmql_query='(MlsStatus = SLD), (CloseDate = 2022-01-01-2022-05-31), (Longitude = {}+), (Longitude = {}-), (Latitude = {}+), (Latitude = {}-), (PostalCode = {}),('.format(lo_left, lo_right, la_left, la_right, zip))
  lis = []
  count = 0
  di ={}
  df = pd.DataFrame()
  for item in result:
    if(item['PropertyType'] == 'Residential Lease' and item['PropertySubType'] == 'Single Family Residence'):
            df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True) #?
            count += 1
            di = item # 存表头
  for key in di:
    lis.append(key)
  if(count >= 5): 
    df = df.T
    df.columns = lis
    return df
  else:
    lo_left -= 0.008
    lo_right += 0.008
    la_left -= 0.008
    la_right += 0.008

# my code__________
    if(a ==0):
      bathroom_left -= 1
      bathyroom_right +=1
    a+=1


    return lease_findifNull(lo_left, lo_right, la_left, la_right, research, zip,a)


In [ ]:
def lease_findHouse(house_leftLa,house_rightLa,house_leftLo,house_rightLo,house_leftSqt,house_rightSqt,research,zip,a,column):
  result = research.search(resource = 'Property', resource_class='Property', dmql_query='(MlsStatus = SLD), (CloseDate = 2022-01-01-2022-05-31), (Longitude = {}+), (Longitude = {}-), (Latitude = {}+), (Latitude = {}-), (PostalCode = {}),(LivingArea = {}+),(LivingArea = {}-)'.format(house_leftLo, house_rightLo, house_leftLa, house_rightLa, zip,house_leftSqt,house_rightSqt))
  basic = house_leftSqt
  count = 0
  df = pd.DataFrame()
  for item in result:
    if(item['PropertyType'] == 'Residential Lease' and item['PropertySubType'] == 'Single Family Residence'):
            df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True) #?
            count += 1
          
    if(count >= 5):  
      df = df.T
      df.columns = column # 
    return df
  else:
    house_leftLo -= 0.008
    house_rightLo += 0.008
    house_leftLa -= 0.008
    house_rightLa += 0.008

    #living area
    if(a == 0):
      if(basic <= 1000):
        house_leftSqt -=100
        house_rightSqt +=100
      elif(basic <= 2200):
        house_leftSqt -=200
        house_rightSqt +=200
      else:
        house_leftSqt -=300
        house_rightSqt +=300
      a += 1

  return lease_findHouse(house_leftLa,house_rightLa,house_leftLo,house_rightLo,house_leftSqt,house_rightSqt,research,zip,a,column)

In [ ]:
def findfirst(result):
    count = 0
    for item in result:
        count += 1
    if(count == 0):
        return False
    else:
        return True    

In [ ]:
def insertId(pulled_value, mls,rets_client):
  column = []
  for key in pulled_value:
    column.append(key)
  print("\nThis value exists in Website")
  zip = pulled_value.get('PostalCode')
  house_leftLa = float(pulled_value.get('Latitude'))
  house_rightLa = float(pulled_value.get('Latitude')) 
  house_leftLo = float(pulled_value.get('Longitude'))
  house_rightLo = float(pulled_value.get('Longitude'))
  house_leftSqt = float(pulled_value.get('LivingArea'))
  house_rightSqt = float(pulled_value.get('LivingArea'))
  house_bed = pulled_value.get("BedroomsTotal")
  house_bath = pulled_value.get("BathroomsTotalInteger")
  a = 0
  result1 = lease_findHouse(house_leftLa,house_rightLa,house_leftLo,house_rightLo,house_leftSqt,house_rightSqt,rets_client,zip,a,column)        
  result1 = result1[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
  'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
  'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
  'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
  'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
  'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
  'USProperty_MUI', 'YearBuilt']]
  price=[]
  for item in result1['ClosePrice']:
    item = float(item)
    price.append(item)
  result1['ClosePrice'] = price
  mean1 = result1['ClosePrice'].mean()
  top1 = list(result1['ClosePrice'])
  top1.sort(reverse = True)
  top1_mean = sum(top1[0:3])/3
  print('The mean for {} is {}'.format(mls, mean1))
  print('The top 3 for {} are {}'.format(mls, top1[0:3]))
  print('The top 3 Avg for {} are {}'.format(mls, top1_mean))
  return result1.to_csv('{}.csv'.format(mls))

In [ ]:
def findAdd(address, city, state, zip):
  result = cg.address(address, city = city, state= state,zipcode = zip)
  lo_left = result[0]['coordinates']['x']
  lo_right = result[0]['coordinates']['x']
  la_left = result[0]['coordinates']['y']
  la_right = result[0]['coordinates']['y']
  result = lease_findifNull(lo_left, lo_right, la_left, la_right, rets_client, zip)
  result = result[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
  'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
  'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
  'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
  'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
  'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
  'USProperty_MUI', 'YearBuilt']]

  price = []
  for item in result['ClosePrice']:
    item = float(item)
    price.append(item)
  result['ClosePrice'] = price
  mean = result['ClosePrice'].mean()
  top = list(result['ClosePrice'])
  top.sort(reverse = True)
  top_mean = sum(top[0:3])/ 3 # new added
  print('The Avg for {}, {}, {} is {}'.format(address, state, zip, mean))
  print('The top 3 for {}, {}, {} are {}'.format(address, state, zip, top[0:3]))
  print('The top 3 Avg for {}, {}, {} is {}').format(address, state, zip, top_mean) #new added
  return result.to_csv('{},{},{}.csv'.format(address, state, zip))


    

In [ ]:
def get(i,finalN,number,code,rets_client,street,city,state):
  pulled_value = {}
  result = rets_client.search(resource = 'Property', resource_class='Property', dmql_query='(StreetNumber = {}),(StreetName = {}),(PostalCode = {})'.format(number[i],finalN[i],code[i]))
  for item in result:
    pulled_value = item
  if(len(pulled_value) != 0):
    column = []
    for key in pulled_value:
      column.append(key)
      print("\nThis value exists in Website")
      zip = pulled_value.get('PostalCode')
      house_leftLa = float(pulled_value.get('Latitude'))
      house_rightLa = float(pulled_value.get('Latitude')) 
      house_leftLo = float(pulled_value.get('Longitude'))
      house_rightLo = float(pulled_value.get('Longitude'))
      house_leftSqt = float(pulled_value.get('LivingArea'))
      house_rightSqt = float(pulled_value.get('LivingArea'))
      a = 0

      result1 = lease_findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, rets_client, column, zip, a)        
      result1 = result1[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]
      price = []
      for item in result1['ClosePrice']:
        item = float(item)
        price.append(item)
      result1['ClosePrice'] = price
      mean1 = result1['ClosePrice'].mean()
      top1 = list(result1['ClosePrice'])
      top1.sort(reverse = True)
      top1_mean = sum(top1[0:3])/3
      return result1.to_csv('{} {} {}.csv'.format(number[i],finalN[i],code[i])), mean1, top1[0:3],top1_mean
    else:
      print('check coordinates')
      result = cg.address(street[i], city = city[i], state= state[i], zipcode = code[i])
      if(len(result) != 0):
        lo_left = result[0]['coordinates']['x']
        lo_right = result[0]['coordinates']['x']
        la_left = result[0]['coordinates']['y']
        la_right = result[0]['coordinates']['y']
        result = lease_findifNull(lo_left, lo_right, la_left, la_right, rets_client, code[i])
        result = result[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]

        price = []
        for item in result['ClosePrice']:
          item = float(item)
          price.append(item)
        result['ClosePrice'] = price
        mean = result['ClosePrice'].mean()
        top = list(result['ClosePrice'])
        top.sort(reverse = True)
        top_mean = sum(top[0:3])/len(top)
        return result.to_csv('{} {} {}.csv'.format(number[i],finalN[i],code[i])), mean, top[0:3],top_mean
      else:
        print('invalid address')
        mean = 0
        top = 0
        return 0, mean, top


In [ ]:
def judge(i):
    if i % 10 == 1:
        return 'st'
    elif i % 10 == 2:
        return 'nd'
    elif i % 10 == 3:
        return 'rd'
    else:
        return 'th'

In [ ]:
def main():
  login_url = 'https://ntrdd.mlsmatrix.com/rets/Login.ashx' 
  username = '0671181_NID'
  password = 'Rt$tg6jx'
  rets_client = Session(login_url, username, password)
  rets_client.login()
  while True:
        print('You want to insert values or a csv, please either insert csv or value!')
        choice = input()
        if choice == 'csv':
            while True:
                print('select which csv you want to use')
                csv = input()
                try:
                    data = pd.read_csv('{}.csv'.format(csv))        
                    code = []
                    name = []
                    number = []
                    street = []
                    city = []
                    state = []
                    for item in data['地址']:
                        a = item.split(',')[0]
                        b = item.split(',')[2]
                        c = item.split(',')[1]
                        street.append(a)
                        city.append(c)
                        numbers = a.split()[0]
                        names = a.split()[1:(len(a.split()) - 1)]
                        codes = b.split()[1][0:5]
                        states = b.split()[0]
                        state.append(states)
                        code.append(codes)
                        name.append(names)
                        number.append(numbers)
                    finalN = []
                    for item in name:
                        string = ""
                        for items in item:
                            string += items
                            string += ' '
                        finalN.append(string[0:len(string) - 1])            
                    for i in range(len(code)):
                        result, mean, top = get(number, finalN, code, i, rets_client, street, city, state)
                        print('The mean for {}, {}, {} is {}'.format(number[i],finalN[i],code[i], mean))
                        print('The top 3 for {}, {}, {} are {}'.format(number[i],finalN[i],code[i], top))
                    return
                except FileNotFoundError:
                    print("Oops! There is no file!  Try again...")
        elif(choice == 'value'):
            while True:
                print('How many mls you want to check?')
                number = input()
                try:
                    Id_list = []
                    for i in range(int(number)):
                        print('Enter the {}{} #MLS'.format(i+1, judge(i+1)))
                        mls = input()
                        Id_list.append(mls)
           #14730621
                    for i in range(len(Id_list)):
                        pulled_value = {}
                        result = rets_client.search(resource='Property', resource_class='Property', dmql_query='(ListingId = {})'.format(Id_list[i]))
                        for items in result:
                            pulled_value = items
                        if(len(pulled_value) != 0):
                            insertId(pulled_value, Id_list[i]) 
                            if(i == int(number) - 1):
                                return
                        else:

                            print('Id {} not exists, please insert specific value'.format(Id_list[i]))
                            print("If you don't know the address, please insert skip")
                            deci = input()
                            if(deci == 'skip'):
                                if(i == int(number) - 1):
                                    return
                                continue
                                
                            else:
                                print('Enter address')
                                address = input()
        #'8615 Barclay Street'
                            
                                print('Enter city')
                                city = input()
        #'Dallas'
                                print('Enter state')
                                state = input()
        #'Texas'
                                print('Enter zipcode')
                                zip = input()
        #'75227'
                                try:
                                    findAdd(address, city, state, zip)
                                    if(i == int(number) - 1):
                                        return
                                except IndexError:
                                    print("invalid input, please next value!")                            
                except ValueError:
                    print("invalid input, try again!")
        else:
            print("invalid input, try again!")

In [ ]:
if __name__ == "__main__":
    print(main())
    

You want to insert values or a csv, please either insert csv or value!
value
How many mls you want to check?
1
Enter the 1st #MLS
14746364

This value exists in Website


NameError: ignored